In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
from tkinter import *
from tkinter import ttk
from turtle import *

In [24]:
# paramètres
#taille monde:

n = 5
m = 7

#taux d'infection :

rzero = 2

#durée de vie virus et vaccin :

dureeinfection = 3
dureevaccin = 12

#taux vaccinal :

taux_immun = 0.8



In [25]:
#génère un monde de taille n,m

def generate_spatial_world(n,m):
    return np.random.choice([0,None],size=(n,m),p = [0.9,0.1])
worldtest = generate_spatial_world(n,m)
worldtest

array([[0, 0, 0, 0, 0, None, 0],
       [0, 0, 0, 0, 0, 0, None],
       [0, 0, None, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [None, 0, None, 0, 0, 0, None]], dtype=object)

In [26]:
#génère la premiere infection aléatoire d'un monde de taille n,m

def infection_world(world):
    while True:
        n = random.randint(0,len(world[:,0])-1)
        m = random.randint(0,len(world[0,:])-1)
        if world[n,m] == 0:
            world[n,m] = 1
            break
    return world

worldtest2=infection_world(worldtest)

In [27]:
#infecte la cellule x,y, si elle n'est pas vaccinée

def infection_cellule(world,x,y):
    worldbis = np.copy(world)
    if world[x,y] != 2:
        worldbis[x,y] = 1
    return worldbis

In [52]:
def infection_vaccin(world,parallele,x,y,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    if world[x,y] ==2:
        chance = random.random()
        if chance >= taux_immun:
            worldbis[x,y] = 1
            parabis[x,y]= 0
    return worldbis,parabis

In [53]:
# idée pour la gestion des temps d'infection /vaccin , création d'un deuxieme vecteur 2d comportant les durées de vie des états actuels

#création d'un monde parallele:

def creation_parallele(world):
    parallele = np.zeros((len(world[:,0]),len(world[0,:])))
    return parallele

#test
#print(creation_parallele(worldtest))
#print(worldtest)
#paralleletest = creation_parallele(worldtest2)

In [54]:

#création d'un monde parallele:

def desinfection(world,duree_infection,parallele):
    for i in range(len(world[:,0])):
        for j in range(len(world[0,:])):
            if (world[i][j] == 1 and parallele[i][j] != duree_infection):
                parallele[i][j] += 1
            elif(parallele[i][j] == duree_infection):
                world[i][j] = 0
                parallele[i][j] = 0
    return world , parallele

#test
#ix = 0
#for ix in range (4):
    #worldtest2,paralleletest = desinfection(worldtest2,3,paralleletest )
    #print(worldtest2,paralleletest)
#print(desinfection(worldtest2,duréeinfection,creation_parallele(worldtest2)))

In [55]:
#génère un tableau en 1d du voisinage de la case nm

def voisins(world,n,m):
    voisins=[]
    for i in range(0,len(world[:,0])):
        for j in range(0,len(world[0,:])):
            if max(abs(i-n),abs(j-m))<=1 and not(i==n and j == m) :
                voisins.append(world[i,j])
    return voisins

In [56]:
#vérifie si il y autre chose que des malades et cases vides dans le voisinage

def verif_voisins(world,n,m):
    for i in voisins(world,n,m):
        if i != None and i!= 1:
            return True
    return False

In [57]:
#fais évoluer le monde en effectuant un tour( pour le moment seul les nouvels infections sont gérées)
#avec la proba
def tour(world,rzero,duree_infection,parallele):
    worldbis = np.copy(world)
    for i in range (0, len(world[:,0])):
        for j in range ( 0, len(world[0,:])):
            if world[i,j] == 1 and verif_voisins(worldbis,i,j):
                nb = 0 
                while nb < rzero and verif_voisins(worldbis,i,j):
                    x = i + (random.randint(-1,1))
                    y = j + (random.randint(-1,1))
                    if (x >= 0 and x <= len(world[:,0])-1) and (y>= 0 and y<= len(world[0,:])-1):
                        if worldbis[x,y] != None and worldbis[x,y]!= 1:
                            worldbis = infection_cellule(worldbis,x,y)
                            nb += 1
    return desinfection(worldbis,duree_infection,parallele)


In [58]:
#effectue le deplacement d'un membre du monde choisi de manière aléatoire
#tp
def deplacement(world,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    x1 = random.randint(0,len(world[:,0])-1)
    y1 = random.randint(0,len(world[0,:])-1)
    x2 = random.randint(0,len(world[:,0])-1)
    y2 = random.randint(0,len(world[0,:])-1)  
    while world[x1,y1] == None or (x1 == x2 and y1 == y2):
        x1 = random.randint(0,len(world[:,0])-1)
        y1 = random.randint(0,len(world[0,:])-1) 
    worldbis[x2,y2] = world[x1,y1]
    worldbis[x1,y1] = world[x2,y2]
    parabis[x2,y2] = parallele[x1,y1]
    parabis[x1,y1] = parallele[x2,y2]
    return worldbis,parabis

In [97]:
#effectue x tour d'évolution du monde en ajoutant un déplacement aléatoire tous les freq_depla tours 

def tour_boucle(world,rzero,nbtour,freq_depla,duree_infection,parallele):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    i = 0
    while i < nbtour:
        if i%freq_depla == 0:
            worldbis,parabis = deplacement(worldbis,parabis)
        worldbis,parabis = tour(worldbis,rzero,duree_infection,parabis)
        #print(worldbis,"\n",parabis,"\n","\n")
        i+= 1
    return worldbis

In [98]:
def vaccin(world):
    worldbis = np.copy(world)
    breack = 0
    for i in range (0, len(world[:,0])):
        if breack != 0:
            break
        for j in range ( 0, len(world[0,:])):
            if world[i][j] == 0 :
                breack +=1
                break
    if breack == 0:
        return worldbis
    else:
        x1 = random.randint(0,len(world[:,0])-1)
        y1 = random.randint(0,len(world[0,:])-1) 
        while world[x1,y1] != 0:
            x1 = random.randint(0,len(world[:,0])-1)
            y1 = random.randint(0,len(world[0,:])-1) 
        worldbis[x1,y1] = 2
        return worldbis

vaccin(worldtest)

array([[0, 0, 0, 0, 0, None, 0],
       [0, 0, 0, 0, 0, 0, None],
       [0, 0, None, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [None, 2, None, 0, 0, 0, None]], dtype=object)

In [99]:
def devaccin(world,parallele,duree_vaccin):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range(len(world[:,0])):
        for j in range(len(world[0,:])):
            if (world[i][j] == 2 and parallele[i][j] != duree_vaccin):
                parabis[i][j] += 1
            elif(world[i][j] == 2 and parallele[i][j] == duree_vaccin):
                worldbis[i][j] = 0
                parabis[i][j] = 0
    return worldbis , parabis

In [123]:
def tour_vaccin(world,rzero,duree_infection,parallele,duree_vaccin,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    for i in range (0, len(world[:,0])):
        for j in range ( 0, len(world[0,:])):
            if world[i,j] == 1 and verif_voisins(worldbis,i,j):
                nb = 0 
                while nb < rzero and verif_voisins(worldbis,i,j):
                    x = i + (random.randint(-1,1))
                    y = j + (random.randint(-1,1))
                    if (x >= 0 and x <= len(world[:,0])-1) and (y>= 0 and y<= len(world[0,:])-1):
                        if worldbis[x,y] == 0:
                            worldbis = infection_cellule(worldbis,x,y)
                            nb += 1
                        if worldbis[x,y] == 2:
                            worldbis,parabis = infection_vaccin(worldbis,parabis,x,y,taux_immun)
                            nb+=1
                worldbis,parabis = desinfection(worldbis,duree_infection,parallele)
    return devaccin(worldbis,parabis,duree_vaccin)



In [124]:
def tour_boucle_vaccin(world,rzero,nbtour,freq_depla,duree_infection,parallele,duree_vaccin,freq_vaccin,taux_immun):
    worldbis = np.copy(world)
    parabis = np.copy(parallele)
    i = 0
    while i < nbtour:
        if i%freq_depla == 0:
            worldbis,parabis = deplacement(worldbis,parabis)
        if i%freq_vaccin == 0:
            worldbis = vaccin(worldbis)
        worldbis,parabis = tour_vaccin(worldbis,rzero,duree_infection,parabis,duree_vaccin,taux_immun)
        print(worldbis,"\n",parabis,"\n","\n")
        i+= 1
    return worldbis

In [125]:
def cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection,duree_vaccin,freq_vaccin,taux_immun):
    world = generate_spatial_world(n,m)
    parallele = creation_parallele(world)
    
    world = infection_world(world)
    worldeux = np.copy(world)
    paradeux = creation_parallele(world)
    
    world,parallele = desinfection(world,duree_infection,parallele)
    worldeux,paradeux = desinfection(worldeux,duree_infection,paradeux)
    
    world = tour_boucle(world,rzero,nbtour,freq_depla,duree_infection,parallele)
    worldeux = tour_boucle_vaccin(worldeux,rzero,nbtour,freq_depla,duree_infection,paradeux,duree_vaccin,freq_vaccin,taux_immun)
    return world , worldeux
    
cycle_entier(n,m,10,5,rzero,dureeinfection,dureevaccin,3,taux_immun)

def print_cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection):
    print(cycle_entier(n,m,nbtour,freq_depla,rzero,duree_infection,parallele))


[[None 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 None 0 0 0]
 [2 1 0 0 None 0 0]
 [0 1 1 0 0 0 0]] 
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]
 [0. 2. 1. 0. 0. 0. 0.]] 
 

[[None 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 1 0 None 0 0 0]
 [2 0 1 1 None 0 0]
 [1 1 0 0 0 0 0]] 
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 3. 0. 0. 0. 0. 0.]
 [2. 0. 2. 1. 0. 0. 0.]
 [2. 1. 0. 0. 0. 0. 0.]] 
 

[[None 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 None 1 0 0]
 [0 0 1 0 None 0 0]
 [1 1 1 0 0 0 0]] 
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0. 0.]
 [0. 0. 3. 0. 0. 0. 0.]
 [1. 2. 1. 0. 0. 0. 0.]] 
 

[[None 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 None 0 0 0]
 [1 1 0 0 None 0 0]
 [1 1 1 1 0 0 2]] 
 [[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [3. 1. 0. 0. 0. 0. 0.]
 [2. 3. 2. 1. 0. 0. 1.]] 
 

[[None 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 None 0 0 0]
 [0 0 1 1 None 0 0]
 [1 0 1 0 1 0 2]] 
 [[0. 0. 0. 0. 0

In [122]:
#essaie tkinter
def fenetre():
    fenetre = Tk()
    fenetre.geometry('800x800')
    fenetre.title('Sars-CoV2')
    fenetre['bg'] = 'black'
    start = Button(fenetre, text ="Simuler" , command =lambda: print_cycle_entier(n,m,10,5,2))
    start.pack()
    end = Button(fenetre ,text = "FERMER" , command = fenetre.destroy)
    end.pack()
    fenetre.mainloop()
fenetre()
